In [5]:
import numpy as np
from sklearn.model_selection import KFold
from src.run import train, test, test_rule
path_checkpoint = "peptides_checkpoint.pt"
import pandas as pd
import requests
from io import BytesIO
from zipfile import ZipFile
import os
from src.util import load_checkpoint, save_checkpoint, compute_metrics
from src.models import LocalModel
from src.data.datasets import UCIAnticancerDataset
from src.data.util import construct_data,construct_loader     
from torch.utils.data import DataLoader
# compute_metrics
from sklearn.metrics import accuracy_score, balanced_accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from src.data.datasets import SequenceDataset
def ovotest(rule, test_loader):
    labels = [0.0, 1.0]
    x_test = list(map(lambda data: data[0].numpy(), test_loader.dataset))
    y_test = list(map(lambda data: data[1].item(), test_loader.dataset))

    y_pred, rules_triggered = rule.batch_apply_rule(x_test)
    y_pred = list(map(float, y_pred))
    res = compute_metrics(y_test, y_pred, labels)
    return y_pred
        

df = pd.read_csv("dataset\question.csv")
batch_size=100
# 5-fold cross validation

dataset = UCIAnticancerDataset(df)
data, y, dictionary = dataset.get_data(df)
kf = KFold(n_splits=5, shuffle=True)
dic={}
for train_index, test_index in kf.split(dataset):

    X_train, X_test = np.array(data)[train_index], np.array(data)[test_index]
    y_train, y_test = np.array(y)[train_index], np.array(y)[test_index]
    test_dataset = SequenceDataset(X_test, y_test, dictionary)
    test_loader = construct_loader(test_dataset, indices=None, batch_size=batch_size)
    
    # devide the data into class number of parts
    class_num = len(set(y_train))
    x_seq_for_each_class = []
    dict_y_train = set(y_train)

    for i in dict_y_train:
        x_seq_for_each_class.append(X_train[y_train == i])
    # generate the train, val with 1-1 combination
    rule_num = 0
    for i in range(0,class_num):
        for j in range(i+1,class_num):
            print("class {} vs class {}".format(i,j) )
            ij_x_train = np.concatenate((x_seq_for_each_class[i], x_seq_for_each_class[j]))
            ij_y_train = np.concatenate((np.zeros(len(x_seq_for_each_class[i])), np.ones(len(x_seq_for_each_class[j]))))
            #  sampler = BucketBatchSampler(np.array(dataset.x, dtype=object), batch_size)
            #  loader = DataLoader(dataset, batch_sampler=sampler, batch_size=1)
            # generate ij_dataset for ij_loader
            ij_dataset = SequenceDataset(ij_x_train, ij_y_train, dictionary)
            loader = construct_loader(dataset, indices=None, batch_size=batch_size)
            # ##########################################################
            input_size = len(dataset.features_name)
            window_size = 3
            base_model_hidden_size = window_size * 2
            base_or_output_size=1
            output_size=1

            pad_border = True

            max_sequence_length =  max([len(x) for x in dataset.x])

            # Compute dimension of convOR input size
            conv_dim_out = max_sequence_length - (window_size - 1)
            if pad_border:
                conv_dim_out = conv_dim_out + (window_size - 1) * 2
            # ##########################################################
            from src.sparsify import Pruning
            pruning = Pruning()
            pruning30 = Pruning(start=30)
            pruning_strategy = pruning30
            epochs = 200
            local_model = LocalModel(input_size, window_size, pad_border, max_sequence_length, base_model_hidden_size, conv_dim_out, base_or_output_size, output_size)
            train(local_model, loader, loader, pruning=pruning_strategy, epochs=epochs, path_checkpoint=path_checkpoint, verbose=True)
            rule = local_model.extract_rule(features_names=dataset.features_name, verbose=False)
            dic[(i,j)]= rule # save the model
            rule_num += len(rule)
            print("rule number: {}".format(len(rule)))
    print("total rule number: {}".format(rule_num))
    # test
    vote=np.zeros([len(X_test),class_num])

    for i in range(0,class_num):
        for j in range(i+1,class_num):
            rule = dic[(i,j)]
            res = ovotest(rule, test_loader)






Distrib 0.48265895953757226 835.0 1730
Distrib 0.48265895953757226 835.0 1730
class 0 vs class 1
Number of batches: 92


C:\Users\14153\AppData\Local\Temp\ipykernel_101784\2290588576.py:40: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  X_train, X_test = np.array(data)[train_index], np.array(data)[test_index]
C:\Users\14153\AppData\Local\Temp\ipykernel_101784\2290588576.py:40: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train, X_test = np.array(data)[train_index], np.array(data)[te

In [1]:
print(rule)

NameError: name 'rule' is not defined